# Illuminating Cognizance: A Comprehensive Look Into Major Power Outages in the U.S.

**Name(s)**: Phu Dang

**Website Link**: (your website link)

## Code

In [1]:
# Importing packages and libraries

import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [32]:
# Adjust dataframe display options to view full output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# pd.set_option('display.max_rows', None)

# Reset to default, comment-out 2 (or 3) lines above and 
    # uncomment 2 lines below, run all before pushing to GitHub
    # Reset to default helps limit the display scope of output dataframes, 
    # easier to navigate notebook
# pd.set_option('display.max_columns', 20)
# pd.set_option('display.width', 80)

## Introduction

Analysis questions of interest:
1. Are there any possible connections between outage duration and regional consumption information?
    - Look into different types of information: price vs. consumption vs. customers served
    - <b>(Personal favorite)</b> I plan to investigate this thoroughly
    <br></br>
2. Is there a relation between anomaly level and cause category?

3. Do the start times of outages seem to affect outage duration?
    - Rationale: A sudden outage at night can give workers more time and space to fix overnight

4. What are some common characteristics of longer outages?

### Cleaning and EDA

In [56]:
# Importing dataset

path = os.path.join('data', 'outage.csv').replace('\\', '/')

df = pd.read_csv(path, header=5, skiprows=[6])
df = df.loc[:, ~df.columns.isin(['OBS', 'variables'])]
df.head()

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",5:00:00 PM,"Sunday, July 3, 2011",8:00:00 PM,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,2332915.0,2114774.0,2113291.0,6562520.0,35.549073,32.225029,32.202431,2308736,276286,10673,2595696,88.9448,10.6440,0.4112,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
1,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",6:38:00 PM,"Sunday, May 11, 2014",6:39:00 PM,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,1586986.0,1807756.0,1887927.0,5284231.0,30.032487,34.210389,35.727564,2345860,284978,9898,2640737,88.8335,10.7916,0.3748,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",8:00:00 PM,"Thursday, October 28, 2010",10:00:00 PM,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,1467293.0,1801683.0,1951295.0,5222116.0,28.097672,34.501015,37.365983,2300291,276463,10150,2586905,88.9206,10.6870,0.3924,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",4:30:00 AM,"Wednesday, June 20, 2012",11:00:00 PM,severe weather,thunderstorm,NaN,2550.0,NaN,68200.0,11.79,9.25,6.71,9.19,1851519.0,1941174.0,1993026.0,5787064.0,31.994099,33.543330,34.439329,2317336,278466,11010,2606813,88.8954,10.6822,0.4224,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",2:00:00 AM,"Sunday, July 19, 2015",7:00:00 AM,severe weather,NaN,NaN,1740.0,250.0,250000.0,13.07,10.16,7.74,10.43,2028875.0,2161612.0,1777937.0,5970339.0,33.982576,36.205850,29.779498,2374674,289044,9812,2673531,88.8216,10.8113,0.3670,54431,49844,1.092027,1.7,4873,292023,1.668704,2.2,5489594,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743


In [57]:
df.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT

In [58]:
# Change postal code column name to STATE.ABBR (state abbreviation)

df.rename(columns={'POSTAL.CODE': 'STATE.ABBR'}, inplace=True)

In [59]:
df.head()

,YEAR,MONTH,U.S._STATE,STATE.ABBR,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",5:00:00 PM,"Sunday, July 3, 2011",8:00:00 PM,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,2332915.0,2114774.0,2113291.0,6562520.0,35.549073,32.225029,32.202431,2308736,276286,10673,2595696,88.9448,10.6440,0.4112,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
1,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",6:38:00 PM,"Sunday, May 11, 2014",6:39:00 PM,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,1586986.0,1807756.0,1887927.0,5284231.0,30.032487,34.210389,35.727564,2345860,284978,9898,2640737,88.8335,10.7916,0.3748,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",8:00:00 PM,"Thursday, October 28, 2010",10:00:00 PM,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,1467293.0,1801683.0,1951295.0,5222116.0,28.097672,34.501015,37.365983,2300291,276463,10150,2586905,88.9206,10.6870,0.3924,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",4:30:00 AM,"Wednesday, June 20, 2012",11:00:00 PM,severe weather,thunderstorm,NaN,2550.0,NaN,68200.0,11.79,9.25,6.71,9.19,1851519.0,1941174.0,1993026.0,5787064.0,31.994099,33.543330,34.439329,2317336,278466,11010,2606813,88.8954,10.6822,0.4224,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",2:00:00 AM,"Sunday, July 19, 2015",7:00:00 AM,severe weather,NaN,NaN,1740.0,250.0,250000.0,13.07,10.16,7.74,10.43,2028875.0,2161612.0,1777937.0,5970339.0,33.982576,36.205850,29.779498,2374674,289044,9812,2673531,88.8216,10.8113,0.3670,54431,49844,1.092027,1.7,4873,292023,1.668704,2.2,5489594,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743


In [60]:
df.dtypes

YEAR                         int64
MONTH                      float64
U.S._STATE                  object
STATE.ABBR                  object
NERC.REGION                 object
CLIMATE.REGION              object
ANOMALY.LEVEL              float64
CLIMATE.CATEGORY            object
OUTAGE.START.DATE           object
OUTAGE.START.TIME           object
OUTAGE.RESTORATION.DATE     object
OUTAGE.RESTORATION.TIME     object
CAUSE.CATEGORY              object
CAUSE.CATEGORY.DETAIL       object
HURRICANE.NAMES             object
OUTAGE.DURATION            float64
DEMAND.LOSS.MW             float64
CUSTOMERS.AFFECTED         float64
RES.PRICE                  float64
COM.PRICE                  float64
IND.PRICE                  float64
TOTAL.PRICE                float64
RES.SALES                  float64
COM.SALES                  float64
IND.SALES                  float64
TOTAL.SALES                float64
RES.PERCEN                 float64
COM.PERCEN                 float64
IND.PERCEN          

In [61]:
# Combining OUTAGE.START.DATE and OUTAGE.START.TIME

dates_start = df['OUTAGE.START.DATE'] + ' ' + df['OUTAGE.START.TIME']
df['OUTAGE.START'] = pd.to_datetime(dates_start)

dates_end = df['OUTAGE.RESTORATION.DATE'] + ' ' + df['OUTAGE.RESTORATION.TIME']
df['OUTAGE.END'] = pd.to_datetime(dates_end)

df.drop(columns=['OUTAGE.START.DATE', 'OUTAGE.START.TIME', \
    'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], inplace=True)

In [62]:
# Get counts of missing values in each column

df.isna().sum()

YEAR                        0
MONTH                       9
U.S._STATE                  0
STATE.ABBR                  0
NERC.REGION                 0
CLIMATE.REGION              6
ANOMALY.LEVEL               9
CLIMATE.CATEGORY            9
CAUSE.CATEGORY              0
CAUSE.CATEGORY.DETAIL     471
HURRICANE.NAMES          1462
OUTAGE.DURATION            58
DEMAND.LOSS.MW            705
CUSTOMERS.AFFECTED        443
RES.PRICE                  22
COM.PRICE                  22
IND.PRICE                  22
TOTAL.PRICE                22
RES.SALES                  22
COM.SALES                  22
IND.SALES                  22
TOTAL.SALES                22
RES.PERCEN                 22
COM.PERCEN                 22
IND.PERCEN                 22
RES.CUSTOMERS               0
COM.CUSTOMERS               0
IND.CUSTOMERS               0
TOTAL.CUSTOMERS             0
RES.CUST.PCT                0
COM.CUST.PCT                0
IND.CUST.PCT                0
PC.REALGSP.STATE            0
PC.REALGSP

In [63]:
df.dtypes

YEAR                              int64
MONTH                           float64
U.S._STATE                       object
STATE.ABBR                       object
NERC.REGION                      object
CLIMATE.REGION                   object
ANOMALY.LEVEL                   float64
CLIMATE.CATEGORY                 object
CAUSE.CATEGORY                   object
CAUSE.CATEGORY.DETAIL            object
HURRICANE.NAMES                  object
OUTAGE.DURATION                 float64
DEMAND.LOSS.MW                  float64
CUSTOMERS.AFFECTED              float64
RES.PRICE                       float64
COM.PRICE                       float64
IND.PRICE                       float64
TOTAL.PRICE                     float64
RES.SALES                       float64
COM.SALES                       float64
IND.SALES                       float64
TOTAL.SALES                     float64
RES.PERCEN                      float64
COM.PERCEN                      float64
IND.PERCEN                      float64


In [74]:
# Adding a duration column in hours

df['DURATION.HR'] = df['OUTAGE.DURATION'] / 60

In [85]:
# Check min and max duration 
print(df['DURATION.HR'].min())
print(df['DURATION.HR'].max())


0.0
1810.8833333333334


In [75]:
df.head()

,YEAR,MONTH,U.S._STATE,STATE.ABBR,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.END,DURATION.HR
0,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,2332915.0,2114774.0,2113291.0,6562520.0,35.549073,32.225029,32.202431,2308736,276286,10673,2595696,88.9448,10.6440,0.4112,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,51.000000
1,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,1586986.0,1807756.0,1887927.0,5284231.0,30.032487,34.210389,35.727564,2345860,284978,9898,2640737,88.8335,10.7916,0.3748,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00,0.016667
2,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,1467293.0,1801683.0,1951295.0,5222116.0,28.097672,34.501015,37.365983,2300291,276463,10150,2586905,88.9206,10.6870,0.3924,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,50.000000
3,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,NaN,2550.0,NaN,68200.0,11.79,9.25,6.71,9.19,1851519.0,1941174.0,1993026.0,5787064.0,31.994099,33.543330,34.439329,2317336,278466,11010,2606813,88.8954,10.6822,0.4224,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,42.500000
4,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,NaN,1740.0,250.0,250000.0,13.07,10.16,7.74,10.43,2028875.0,2161612.0,1777937.0,5970339.0,33.982576,36.205850,29.779498,2374674,289044,9812,2673531,88.8216,10.8113,0.3670,54431,49844,1.092027,1.7,4873,292023,1.668704,2.2,5489594,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00,29.000000


### Univariate Analysis

In [98]:
# Plotting the distribution of outage durations

fig1 = px.histogram(df, x='DURATION.HR', nbins=200, histnorm='probability', \
    title='Distribution of outage duration, in hours')
fig1.update_layout(xaxis_range=[0, 360])
fig1.update_xaxes(title_text='duration in hours')
fig1

In [256]:
# Write to html file

path = os.path.join('assets', 'uni_1.html')
fig1.write_html(path, include_plotlyjs='cdn')

In [97]:
# Sanity check

573 / len(df['DURATION.HR'])

0.37353324641460234

In [101]:
# Plotting the distribution of cause category

fig2 = px.histogram(df, x='CAUSE.CATEGORY', histnorm='probability', \
    title='Distribution of outage cause category')
fig2.update_xaxes(title_text='cause category')
fig2

In [257]:
# Write to html file

path = os.path.join('assets', 'uni_2.html')
fig2.write_html(path, include_plotlyjs='cdn')

### Bivariate Analysis

In [264]:
# Plotting a scatterplot between outage duration and residential electricity price

fig3 = px.scatter(df, x='RES.PRICE', y='DURATION.HR', title='Outage duration and residential electricity price, monthly')
fig3.update_xaxes(title_text='cents/kilowatt-hour')
fig3.update_yaxes(title_text='hours')
fig3

In [258]:
# Write to html file

path = os.path.join('assets', 'bi_1.html')
fig3.write_html(path, include_plotlyjs='cdn')

In [140]:
# Plotting a scatterplot between outage duration and customers served

fig4 = px.scatter(df, x='TOTAL.CUSTOMERS', y='DURATION.HR'\
    , title='Outage duration and total number of customers served, annually')
fig4.update_xaxes(title_text='number of customers served')
fig4.update_yaxes(title_text='hours')
fig4

In [260]:
# Plotting a barplot between outage duration and regional economic output

fig5 = px.scatter(df, x='PC.REALGSP.STATE', y='DURATION.HR'\
    , title='Outage duration and per capita GSP')
fig5.update_xaxes(title_text='per capita real gross state product (measured in 2009 chained U.S. dollars)')
fig5.update_yaxes(title_text='hours')
fig5

In [261]:
# Write to html file

path = os.path.join('assets', 'bi_2.html')
fig5.write_html(path, include_plotlyjs='cdn')

### Interesting Aggregates

In [220]:
# Plot average outage duration for every combination of state and cause category

group1 = pd.pivot_table(df, index=['U.S._STATE'], columns=['CAUSE.CATEGORY'], \
    values='DURATION.HR', aggfunc=np.mean)
group1['dummy'] = group1.sum(axis=1)
group1 = group1.sort_values(by='dummy', ascending=True).drop(columns=['dummy'])
fig6 = group1.plot(kind='barh', \
    title='Average outage duration by state and cause category',
    labels={'U.S._STATE': 'state', 'value': 'average outage duration hours'})
fig6.update_layout(height=1000, width=1500, legend=dict(title='cause category'))
fig6


In [262]:
# Write to html file

path = os.path.join('assets', 'multi_1.html')
fig6.write_html(path, include_plotlyjs='cdn')

In [221]:
# Sanity check

group1.sum(axis=1).sort_values(ascending=False).index

Index(['Michigan', 'Louisiana', 'Wisconsin', 'New York', 'Arizona', 'Indiana',
       'Texas', 'Kentucky', 'California', 'Florida', 'Kansas', 'West Virginia',
       'Iowa', 'Washington', 'New Jersey', 'Pennsylvania', 'Ohio', 'Illinois',
       'Oklahoma', 'Missouri', 'District of Columbia', 'Massachusetts',
       'Tennessee', 'Maryland', 'Arkansas', 'Maine', 'Minnesota', 'Utah',
       'Nebraska', 'South Carolina', 'Colorado', 'Oregon', 'North Carolina',
       'Connecticut', 'Delaware', 'Virginia', 'Idaho', 'New Hampshire',
       'Georgia', 'Alabama', 'Hawaii', 'North Dakota', 'Nevada', 'Mississippi',
       'New Mexico', 'Wyoming', 'Montana', 'South Dakota', 'Vermont'],
      dtype='object', name='U.S._STATE')

In [232]:
# Get aggregated mean outage duration in minutes, hours, and customers affected by state and cause category

pd.set_option('display.max_rows', None)
group2 = df.groupby(by=['STATE.ABBR', 'CAUSE.CATEGORY']).mean()[['OUTAGE.DURATION', 'DURATION.HR', 'CUSTOMERS.AFFECTED']]
group2.rename(columns={'OUTAGE.DURATION': 'Avg duration (mins)', \
    'DURATION.HR': 'Avg duration (hrs)',
    'CUSTOMERS.AFFECTED': 'Avg # of customers affected'})
# pd.set_option('display.max_rows', 20)

Avg duration (mins)  \
STATE.ABBR CAUSE.CATEGORY                                       
AK         equipment failure                              NaN   
AL         intentional attack                       77.000000   
           severe weather                         1421.750000   
AR         equipment failure                       105.000000   
           intentional attack                      547.833333   
           islanding                                 3.000000   
           public appeal                          1063.714286   
           severe weather                         2701.800000   
AZ         equipment failure                       138.500000   
           intentional attack                      639.600000   
           severe weather                        25726.500000   
           system operability disruption           384.500000   
CA         equipment failure                       524.809524   
           fuel supply emergency                  6154.600000   
           intentional attack                      946.458333   
           islanding                               214.857143   
           public appeal                          2028.111111   
           severe weather                         2928.373134   
           system operability disruption           363.666667   
CO         fuel supply emergency                          NaN   
           intentional attack                      117.000000   
           islanding                                 2.000000   
           severe weather                         2727.250000   
           system operability disruption           279.750000   
CT         intentional attack                       49.125000   
           severe weather                         2262.600000   
DC         equipment failure                       159.000000   
           severe weather                         4764.111111   
DE         equipment failure                        50.000000   
           intentional attack                       38.918919   
           severe weather                         2153.500000   
           system operability disruption                  NaN   
FL         equipment failure                       554.500000   
           intentional attack                       50.000000   
           public appeal                          4320.000000   
           severe weather                         6420.192308   
           system operability disruption           205.700000   
GA         intentional attack                      108.000000   
           severe weather                         1422.750000   
HI         severe weather                          997.500000   
           system operability disruption           237.000000   
IA         intentional attack                     5657.800000   
           severe weather                         3353.666667   
ID         intentional attack                      307.500000   
           public appeal                          1548.000000   
           system operability disruption           179.666667   
IL         equipment failure                       149.000000   
           fuel supply emergency                  2761.000000   
           intentional attack                     1450.000000   
           public appeal                           120.000000   
           severe weather                         1650.700000   
IN         equipment failure                         1.000000   
           fuel supply emergency                 12240.000000   
           intentional attack                      421.875000   
           islanding                               125.333333   
           severe weather                         4523.291667   
           system operability disruption          4671.600000   
KS         intentional attack                      561.000000   
           public appeal                           913.000000   
           severe weather                         9346.000000   
KY         e

In [228]:
# sanity check

df[df['STATE.ABBR'] == 'AK']

,YEAR,MONTH,U.S._STATE,STATE.ABBR,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.END,DURATION.HR
1533,2000,NaN,Alaska,AK,ASCC,NaN,NaN,NaN,equipment failure,failure,NaN,NaN,35.0,14273.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230534,38074,854,273530,84.2811,13.9195,0.3122,57401,44745,1.282847,-2.2,724,36046,2.008545,0.2,627963,66.02,21.56,1802.6,1276.0,0.4,0.05,0.02,85.761154,14.238846,2.901182,NaT,NaT,NaN


In [233]:
group1.head()

CAUSE.CATEGORY,equipment failure,fuel supply emergency,intentional attack,islanding,public appeal,severe weather,system operability disruption
U.S._STATE,,,,,,,
Vermont,NaN,NaN,0.590741,NaN,NaN,NaN,NaN
South Dakota,NaN,NaN,NaN,2.000000,NaN,NaN,NaN
Montana,NaN,NaN,1.550000,0.575000,NaN,NaN,NaN
Wyoming,1.016667,NaN,0.005556,0.533333,NaN,1.766667,NaN
New Mexico,NaN,1.266667,2.908333,NaN,NaN,NaN,0.0


In [247]:
# Get pivot table of average anomaly level by cause category and category detail

group2 = pd.pivot_table(df, index=['CAUSE.CATEGORY'], \
    columns=['CAUSE.CATEGORY.DETAIL'], values='ANOMALY.LEVEL', aggfunc=np.mean)
fig7 = group2.plot(kind='barh', \
    title='Average anomaly level by cause category, subset by category '+
    'detail<br><sup>Anomaly level represents the oceanic El Niño/La Niña (ONI)'+
    ' index, estimated as a 3-month running mean of ERSST.v4 SST anomalies in'+
    ' the Niño 3.4 region</sup>',
    labels={'CAUSE.CATEGORY': 'cause category', 'value': 'average anomaly level (ONI Index)'})
fig7.update_layout(legend=dict(title='cause category detail'))
fig7

In [263]:
# Write to html file

path = os.path.join('assets', 'multi_2.html')
fig7.write_html(path, include_plotlyjs='cdn')

In [234]:
df.head()

,YEAR,MONTH,U.S._STATE,STATE.ABBR,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.END,DURATION.HR
0,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,2332915.0,2114774.0,2113291.0,6562520.0,35.549073,32.225029,32.202431,2308736,276286,10673,2595696,88.9448,10.6440,0.4112,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,51.000000
1,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,1586986.0,1807756.0,1887927.0,5284231.0,30.032487,34.210389,35.727564,2345860,284978,9898,2640737,88.8335,10.7916,0.3748,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00,0.016667
2,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,1467293.0,1801683.0,1951295.0,5222116.0,28.097672,34.501015,37.365983,2300291,276463,10150,2586905,88.9206,10.6870,0.3924,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,50.000000
3,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,NaN,2550.0,NaN,68200.0,11.79,9.25,6.71,9.19,1851519.0,1941174.0,1993026.0,5787064.0,31.994099,33.543330,34.439329,2317336,278466,11010,2606813,88.8954,10.6822,0.4224,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,42.500000
4,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,NaN,1740.0,250.0,250000.0,13.07,10.16,7.74,10.43,2028875.0,2161612.0,1777937.0,5970339.0,33.982576,36.205850,29.779498,2374674,289044,9812,2673531,88.8216,10.8113,0.3670,54431,49844,1.092027,1.7,4873,292023,1.668704,2.2,5489594,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00,29.000000


### Assessment of Missingness

In [253]:
# Inspect the outages where month is missing (likely MAR, explained by year)

df[df['MONTH'].isna()].head(11)

,YEAR,MONTH,U.S._STATE,STATE.ABBR,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.END,DURATION.HR
239,2000,NaN,Texas,TX,FRCC,South,NaN,NaN,equipment failure,transformer outage,NaN,NaN,46.0,43000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8023266,1093414,61280,9299829,86.2733,11.7574,0.6589,45102,44745,1.007979,1.7,30908,944631,3.271965,10.3,20944499,84.70,9.35,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873,NaT,NaT,NaN
339,2000,NaN,Alabama,AL,SERC,Southeast,NaN,NaN,severe weather,thunderstorm,NaN,NaN,NaN,160000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1930037,313017,6252,2262753,85.2960,13.8335,0.2763,33712,44745,0.753425,1.1,5704,150090,3.800386,2.2,4452173,59.04,10.39,1278.5,988.7,40.4,4.36,0.99,96.613888,3.386112,2.018314,NaT,NaT,NaN
365,2000,NaN,Illinois,IL,SERC,Central,NaN,NaN,severe weather,wildfire,NaN,NaN,NaN,11000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4748863,493670,5009,5282401,89.8997,9.3456,0.0948,49276,44745,1.101263,2.8,15815,612709,2.581160,5.8,12434161,88.49,8.52,2877.6,1759.5,28.6,7.11,1.12,95.864558,4.135442,1.415893,NaT,NaT,NaN
766,2000,NaN,North Carolina,NC,SERC,Southeast,NaN,NaN,severe weather,thunderstorm,NaN,NaN,175.0,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3561203,513727,12577,4105711,86.7378,12.5125,0.3063,42201,44745,0.943144,1.5,8262,341051,2.422512,2.2,8081614,66.09,11.21,1367.2,1043.7,73.5,9.48,2.11,90.336127,9.663873,7.528940,NaT,NaT,NaN
887,2000,NaN,Delaware,DE,RFC,Northeast,NaN,NaN,system operability disruption,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,335282,40616,553,377219,88.8826,10.7672,0.1466,64013,44745,1.430618,2.1,1016,50338,2.018356,0.4,786373,83.30,14.59,1838.3,1083.0,97.3,20.88,6.21,78.304540,21.695460,3.656087,NaT,NaT,NaN
1318,2000,NaN,Virginia,VA,SERC,Southeast,NaN,NaN,equipment failure,relaying malfunction,NaN,NaN,143.0,37000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2767245,306821,5371,3122909,88.6111,9.8248,0.1720,47456,44745,1.060588,2.5,6978,337211,2.069328,1.8,7105817,75.45,5.66,2265.2,1179.2,53.3,6.75,0.97,92.320281,7.679719,2.997078,NaT,NaT,NaN
1506,2002,NaN,Kansas,KS,SPP,South,NaN,NaN,severe weather,winter storm,NaN,NaN,550.0,270000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1157820,194996,12885,1379347,83.9397,14.1368,0.9341,40525,45097,0.898619,1.1,2462,109966,2.238874,1.1,2713535,74.20,24.03,2176.5,1983.0,9.1,1.19,0.42,99.369212,0.632004,0.632004,NaT,NaT,NaN
1530,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,1650.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309997,53709,2331,366037,84.6901,14.6731,0.6368,42913,48909,0.877405,3.5,1019,27868,3.656524,0.7,649422,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT,NaN
1533,2000,NaN,Alaska,AK,ASCC,NaN,NaN,NaN,equipment failure,failure,NaN,NaN,35.0,14273.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230534,38074,854,273530,84.2811,13.9195,0.3122,57401,44745,1.282847,-2.2,724,36046,2.008545,0.2,627963,66.02,21.56,1802.6,1276.0,0.4,0.05,0.02,85.761154,14.238846,2.901182,NaT,NaT,NaN


In [254]:
# Inspect the outages where demand loss is missing (demand loss is likely NMAR)

df[df['DEMAND.LOSS.MW'].isna()].head(11)

,YEAR,MONTH,U.S._STATE,STATE.ABBR,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.END,DURATION.HR
0,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,2332915.0,2114774.0,2113291.0,6562520.0,35.549073,32.225029,32.202431,2308736,276286,10673,2595696,88.9448,10.6440,0.4112,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,51.000000
1,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,1586986.0,1807756.0,1887927.0,5284231.0,30.032487,34.210389,35.727564,2345860,284978,9898,2640737,88.8335,10.7916,0.3748,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00,0.016667
2,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,1467293.0,1801683.0,1951295.0,5222116.0,28.097672,34.501015,37.365983,2300291,276463,10150,2586905,88.9206,10.6870,0.3924,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,50.000000
3,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,NaN,2550.0,NaN,68200.0,11.79,9.25,6.71,9.19,1851519.0,1941174.0,1993026.0,5787064.0,31.994099,33.543330,34.439329,2317336,278466,11010,2606813,88.8954,10.6822,0.4224,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,42.500000
5,2010,11.0,Minnesota,MN,MRO,East North Central,-1.4,cold,severe weather,winter storm,NaN,1860.0,NaN,60000.0,10.63,8.34,6.15,8.28,1676347.0,1786144.0,1909874.0,5374150.0,31.192784,33.235842,35.538160,2300291,276463,10150,2586905,88.9206,10.6870,0.3924,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-11-13 15:00:00,2010-11-14 22:00:00,31.000000
6,2010,7.0,Minnesota,MN,MRO,East North Central,-0.9,cold,severe weather,tornadoes,NaN,2970.0,NaN,63000.0,11.41,9.11,6.71,9.12,2187537.0,2100828.0,2084709.0,6374935.0,34.314656,32.954501,32.701651,2300291,276463,10150,2586905,88.9206,10.6870,0.3924,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-07-17 20:30:00,2010-07-19 22:00:00,49.500000
9,2013,6.0,Minnesota,MN,MRO,East North Central,-0.2,normal,severe weather,hailstorm,NaN,3621.0,NaN,400000.0,12.71,10.31,7.29,10.00,1688619.0,1895099.0,1905375.0,5490631.0,30.754553,34.515140,34.702296,2329734,281187,11383,2622305,88.8430,10.7229,0.4341,52479,48396,1.084366,1.7,5613,284542,1.972644,2.3,5420541,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2013-06-21 17:39:00,2013-06-24 06:00:00,60.350000
10,2013,6.0,Minnesota,MN,MRO,East North Central,-0.2,normal,severe weather,hailstorm,NaN,7740.0,NaN,193000.0,12.71,10.31,7.29,10.00,1688619.0,1895099.0,1905375.0,5490631.0,30.754553,34.515140,34.702296,2329734,281187,11383,2622305,88.8430,10.7229,0.4341,52479,48396,1.084366,1.7,5613,284542,1.972644,2.3,5420541,73.27,

### Hypothesis Testing

In [ ]:
# TODO